In [1]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [6]:
import numpy as np
import itertools

def tfm_tensors(doc):
    import numpy as np

    tfm_str = doc._.trf_data.wordpieces.strings
    tfm_str = list(itertools.chain(*tfm_str))
    
    token_emb, sent_emb = doc._.trf_data.tensors
    print(token_emb.shape)
    token_emb = token_emb.reshape(-1, token_emb.shape[-1])
    #print(token_emb.shape, len(tfm_str[0]))
    n_dims = token_emb.shape[-1]

    vs = []
    for x in doc._.trf_data.align:
        #print([tfm_str[i[0]] for i in x.data])
        v = []
        for i in x.data:
            #print("?", token_emb.shape[0], i[0])
            v.append(token_emb[i[0]])
        if v:
            v = np.average(np.array(v), axis=0)
        else:
            v = np.zeros((n_dims))
        #print(v.shape, v[:5])
        vs.append(v)
        
    return np.array(vs)

def sim(doc1, s1, doc2, s2):
    v1 = tfm_tensors(doc1)
    v2 = tfm_tensors(doc2)
    i = [str(s) for s in doc1].index(s1)
    j = [str(s) for s in doc2].index(s2)
    return np.dot(v1[i] / np.linalg.norm(v1[i]), v2[j] / np.linalg.norm(v2[j]))

In [7]:
#doc = nlp("Wise Children’s insouciance is near stoic. A book about old ladies is, helplessly, going to be a book about ‘the way of all flesh’. ‘Whence came we? Whither goeth we? I know the answer to the second question, of course. Bound for oblivion, nor leave a wrack behind.’ It’s a limelight novel that knows the dark, that ‘wars are facts we cannot fuck away’. Its insouciance is its response in the face of tragedy, poverty, illegitimacy, hierarchy, and most grave of all, ‘untimely death.’ What to do? ‘We’ll go on singing and dancing until we drop in our tracks, won’t we kids?’ In her critical writings Carter associates lightness with stoicism more than once. She comments on one of the movie stars she was most fascinated by, Louise Brooks, who, in Pabst’s Pandora’s Box, ‘typifies the subversive violence inherent in beauty and a light heart.’13 In a book as much about the degeneration game as it is about generation and generations, Dora and Nora, in their seventies, at the fag end of the British empire, go to a broken-down old cinema and see the film of their young selves in A Midsummer Night’s Dream, ‘two batty old tarts with their eyes glued to their own ghosts.’")
#doc = nlp("In that essay on Colette, Carter wrote about the moment in Simone de Beauvoir’s memoirs when she records the surreality of her being in the same room as (if not actually speaking to) Colette.")
with open("/Users/arbeit/Wise Children Mini.txt", "r") as f:
    doc = nlp(f.read())

In [11]:
doc2 = nlp("person")

In [14]:
sim(doc, "communality", doc2, "person")

(1, 43, 768)
(1, 3, 768)


0.5506242087667998

In [10]:
sim(doc, "taken", doc2, "shared")

(1, 43, 768)
(1, 3, 768)


0.4380336470114672

In [61]:
#tfm_tensors(doc).shape

In [5]:
tfm_tensors(doc)[:5, :10]

(2, 146, 768)


array([[-1.14375722, -0.30409056,  0.86509454, -0.31990629,  1.06158209,
         0.85182852,  0.85282427, -0.175081  , -0.36473972, -1.51145494],
       [-0.90024966, -0.7322849 , -0.06842124,  0.73083138,  1.21642399,
         0.43938926,  0.84701651,  1.5145514 ,  1.45519233, -0.8445735 ],
       [-0.56277001, -0.49145812, -0.16693702,  1.56473219,  2.46784616,
         1.90175247, -0.59321785,  0.45290664, -0.14161064,  0.071206  ],
       [-0.37899685, -0.61451423,  0.47324866, -0.12926887,  1.29338622,
        -1.23498452,  0.92027628, -0.1379444 , -1.29010332, -0.41220453],
       [-0.16907969, -0.00849829, -0.05117168,  1.01450431,  0.63966888,
         1.59416902,  1.27664888,  0.54130089,  0.87566674,  0.12214404]])

In [39]:
len(doc._.trf_data.align)

263

In [40]:
len(doc)

263

In [43]:
v = doc._.trf_data.tensors[0]
v.reshape(-1, v.shape[-1]).shape

(495, 768)